# Data Processing: Raw to Intermediate 

Data Source: [Ask A Manager salary survey 2019](https://docs.google.com/spreadsheets/d/1rGCKXIKt-7l5gX06NAwO3pjqEHh-oPXtB8ihkp0vGWo/edit#gid=382484678)

##### Load Libraries

In [22]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import re

In [23]:
# first row does not have useful information so will skip 
aam_salary_raw = pd.read_csv('../data/01_raw/aam_salary_raw.csv', 
                            skiprows=1)

##### Rename all columns to take out spaces

In [24]:
# current column names are long and contain spaces
aam_salary_raw.columns

Index(['Timestamp', 'How old are you?', 'What industry do you work in?',
       'Job title', 'What is your annual salary?',
       'Please indicate the currency',
       'Where are you located? (City/state/country)',
       'How many years of post-college professional work experience do you have?',
       'If your job title needs additional context, please clarify here:',
       'If "Other," please indicate the currency here: '],
      dtype='object')

In [25]:
# simplify column names
aam_salary_raw.head()
aam_salary_raw.rename(columns={"Timestamp": "timestamp", "How old are you?": "age", 
                               "What industry do you work in?": "industry", "Job title": "job_title", 
                              "What is your annual salary?": "annual_salary", 
                               "Please indicate the currency": "currency", 
                              "Where are you located? (City/state/country)": "location", 
                              "How many years of post-college professional work experience do you have?": "work_years", 
                              "If your job title needs additional context, please clarify here:" : "notes", 
                              "If \"Other,\" please indicate the currency here: ": "other_currency"}, 
                     inplace=True)


##### View the datatype of each variable

In [26]:
aam_salary_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30995 entries, 0 to 30994
Data columns (total 10 columns):
timestamp         30995 non-null object
age               30995 non-null object
industry          30171 non-null object
job_title         30993 non-null object
annual_salary     30992 non-null object
currency          30995 non-null object
location          29500 non-null object
work_years        30995 non-null object
notes             7004 non-null object
other_currency    236 non-null object
dtypes: object(10)
memory usage: 2.4+ MB


In [27]:
# change timestamp from object to datetime
aam_salary_raw.timestamp = pd.to_datetime(aam_salary_raw.timestamp)
# erase all commas, dollar signs & spaces
aam_salary_raw.annual_salary = aam_salary_raw.annual_salary.str.replace(',', '')
aam_salary_raw.annual_salary = aam_salary_raw.annual_salary.str.replace('$', '')
aam_salary_raw.annual_salary = aam_salary_raw.annual_salary.str.replace(' ', '')
aam_salary_raw['annual_salary'] = aam_salary_raw['annual_salary'].str.split('.').str[0]

In [28]:
# filter only numbers for the annual_salary variable 
new_sal = []
for salary in aam_salary_raw.annual_salary:
    new_sal.append(re.findall(r'\d+', str(salary)))
aam_salary_raw.annual_salary = new_sal

In [29]:
# 853 entries that have multiple numbers
count = 0
for item in new_sal:
    if len(item) > 1:
        count += 1
print(count)

217


In [30]:
# take the first value from the brakets
aam_salary_raw['annual_salary'] = aam_salary_raw['annual_salary'].str.get(0)
# change datatype to numeric
aam_salary_raw.annual_salary = pd.to_numeric(aam_salary_raw.annual_salary)

In [31]:
aam_salary_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30995 entries, 0 to 30994
Data columns (total 10 columns):
timestamp         30995 non-null datetime64[ns]
age               30995 non-null object
industry          30171 non-null object
job_title         30993 non-null object
annual_salary     30977 non-null float64
currency          30995 non-null object
location          29500 non-null object
work_years        30995 non-null object
notes             7004 non-null object
other_currency    236 non-null object
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 2.4+ MB


##### There are 555 entries with annual salaries below 1,000. I'll drop these entries. Will need to drop these entries in processing. 
```python 
aam_drop = aam_salary_raw.loc[aam_salary_raw['annual_salary'] < 1000]
```

In [32]:
aam_salary_raw.to_csv('../data/02_intermediate/aam_sal_inter.csv', index=False)